In [9]:
import json
import networkx as nx

In [11]:
# Read JSON from a file
with open('edges_W1.json', 'r') as json_file:
    edges_dict = json.load(json_file)

with open('nodes_W1.json', 'r') as json_file:
    nodes_dict = json.load(json_file)


In [12]:


# Create the directed graph from edges
G = nx.DiGraph()

# Add edges with attributes
for source, targets in edges_dict.items():
    for target, edge_attributes in targets.items():
        G.add_edge(source, target, **edge_attributes)

# Add node-specific attributes
for node, attributes in nodes_dict.items():
    G.add_node(node, **attributes)

# Optional: Check the graph structure
print(f"Graph nodes: {G.nodes(data=True)}")
print(f"Graph edges: {G.edges(data=True)}")


Graph nodes: [('kru', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('bs3', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('k4w', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('xtk', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('ngv', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('qx3', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('ywc', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('nrn', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('4yx', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('kmw', {'leftrightself': 0, 'gender': 2.0, 'income': nan}), ('b3p', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('udg', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('ack', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('h9x', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('cyw', {'leftrightself': 0, 'gender': 2.0, 'income': nan}), ('9he', {'leftrightself': 0, 'gender': 1.0, 'income': nan}), ('qew', {'

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the graph
def plot_graph(G):
    # Compute positions for nodes
    pos = nx.kamada_kawai_layout(G)  # Use spring layout for better distribution
    
    # Extract node attributes for styling
    node_colors = [G.nodes[node].get("gender", "gray") for node in G.nodes()]
    color_map = {2: "red", 1: "green", "gray": "gray"}
    node_colors = [color_map.get(color, "gray") for color in node_colors]
    
    node_sizes = [G.nodes[node].get("age", 25) * 10 for node in G.nodes()]  # Scale size by age

    # Extract edge attributes for styling
    edge_weights = [2 if edge[2].get("friend") else 0.5 for edge in G.edges(data=True)]
    
    # Draw the graph
    plt.figure(figsize=(20, 16))
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9)
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.7, edge_color="gray")
    #nx.draw_networkx_labels(G, pos, font_size=10, font_color="black")

    # Add legend for node colors
    handles = [plt.Line2D([0], [0], marker='o', color='w', label=gender, 
                          markerfacecolor=color_map[gender], markersize=10) 
               for gender in color_map]
    plt.legend(handles=handles, title="Gender", loc="best")

    # Display the graph
    plt.title("SS_WS24_W2 Aquaintances Network Gender")
    plt.axis("off")
    plt.show()

# FILEPATH: /Users/ramius/Desktop/CodeVault/01_Project/Work/Susumu/Student_Survey/StudentSurvey_GitDockUpdated/data/SS_Networks.ipynb

# Call the function to plot
plot_graph(G)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import patches

# Define a function to plot the graph with a color gradient for both nodes and edges
def plot_graph_gradient(G):
    # Set the style to dark background
    plt.style.use('dark_background')

    # Compute positions for nodes
    pos = nx.kamada_kawai_layout(G, dim=2, scale=15)  # Use Kamada-Kawai layout for better distribution

    # Get all the 'leftrightself' values and normalize them for nodes
    leftright_values = [G.nodes[node].get("leftrightself", 1) for node in G.nodes()]
    
    # Normalize values to fit within the range 1-11
    norm = mcolors.Normalize(vmin=1, vmax=11)
    cmap = cm.coolwarm  # You can choose any other colormap (e.g., "plasma", "inferno", etc.)

    # Apply the colormap based on the normalized 'leftrightself' values
    node_colors = [cmap(norm(value)) for value in leftright_values]

    # Edge weights (friend-based styling) and color edges based on 'leftright'
    edge_leftright_values = [edge[2].get("leftright", 1) for edge in G.edges(data=True)]
    edge_color_map = [cmap(norm(value)) for value in edge_leftright_values]  # Apply colormap to edges

    # Edge weights based on 'friend' attribute
    edge_weights = [5 if edge[2].get("council") else 1 for edge in G.edges(data=True)]

    # Get node attribute 'income' for node sizes
    income = [G.nodes[node].get("income", 25) for node in G.nodes()]
    # Normalize income values for node sizes
    norm_income = mcolors.Normalize(vmin=min(income), vmax=max(income))
    income = [norm_income(value) * 5000 for value in income]  # Scale for better visualization

    # Draw the graph
    fig, ax = plt.subplots(figsize=(70, 48))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors,node_size=income, alpha=0.9, ax=ax)

    # Draw curved edges using ConnectionPatch
    for edge, weight, color in zip(G.edges(), edge_weights, edge_color_map):
        node1, node2 = edge
        x1, y1 = pos[node1]
        x2, y2 = pos[node2]
        
        # Adjusting curvature
        rad = 0.05  # Modify the rad parameter to control the curvature
        curve = patches.ConnectionPatch([x1, y1], [x2, y2], "data", "data", color=color, linewidth=weight, linestyle='-', connectionstyle=f"arc3,rad={rad}")
        ax.add_patch(curve)

    # Add color bar for the edge gradient
    sm_edge = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm_edge.set_array([])  # Empty array for the color bar
    cbar_edge = plt.colorbar(sm_edge, ax=ax, label="Leftright Value")

    # Display the graph
    plt.title("SS_WS24_W2 Aquaintances Network with Council and LeftRight", color='black', bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.5'))
    plt.axis("off")
    plt.show()

# Call the function to plot
plot_graph_gradient(G)
